In [ ]:
%pylab inline
style.use('jf')
from qutip import *

In [ ]:
import warnings
warnings.simplefilter('default')
np.set_printoptions(linewidth=210)

In [ ]:
from lanczos_timeprop import lanczos_timeprop

In [ ]:
from scipy.sparse import lil_matrix
N = 50000
H = lil_matrix((N,N))
H[range(N),range(N)] = -2.
H[range(N-1),range(1,N)] = 1
H[range(1,N),range(N-1)] = 1
H = H.tocsr()

In [ ]:
class phi(np.ndarray):
    def norm(self):
        return norm(self)
    def dot(self,other):
        return vdot(self,other)

phi0 = (exp(-(arange(N)-N/5)**2/(2*300**2)-1.5j*arange(N)) + exp(-(arange(N)-N/10)**2/(2*50**2)+1j*arange(N))).view(phi)
phi0 /= phi0.norm()

In [ ]:
Hq = Qobj(H)
phi0q = Qobj(phi0)

In [ ]:
prop = lanczos_timeprop(H,phi0,12,1e-10)
prop._step(0,.5)

In [ ]:
ts = linspace(0,100,11)
prop = lanczos_timeprop(H,phi0,12,1e-12)
%time phiLs = prop.propagate(phi0,ts)
%time resQ = sesolve(Hq,phi0q,ts,[],options=Options(atol=1e-8,rtol=1e-6))
phiQs = array([s.full() for s in resQ.states]).squeeze()

In [ ]:
norm(phiLs), norm(phiQs-phiLs), norm(phiQs[-1]-phiLs[-1])

In [ ]:
f,axs = subplots(1,2,figsize=(8,4),sharex=True,sharey=True)
axs[0].pcolormesh(ts,arange(N),abs(phiQs.T)**2)
axs[1].pcolormesh(ts,arange(N),abs(phiLs.T)**2)
axs[0].autoscale(tight=True)
axs[0].set_ylim(4000,11000)
f.tight_layout(pad=0.5)

In [ ]:
convgs = [1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]
qsolve = lambda tol: array([s.full() for s in sesolve(Hq,phi0q,ts,[],options=Options(atol=tol,rtol=tol)).states]).squeeze()
phiLs_convg, phiQs_convg = {}, {}
for c in convgs:
    print(c)
    if c>=1e-14:
        phiLs_convg[c] = lanczos_timeprop(H,phi0,12,c).propagate(phi0,ts)
    print(c)
    phiQs_convg[c] = qsolve(c)

In [ ]:
def err(a,b,ea,eb):
    dd=dict(Q=phiQs_convg,L=phiLs_convg)
    phia = dd[a][ea][-1]
    phib = dd[b][eb][-1]
    return 2*abs(phia-phib)/(abs(phia)+abs(phib)+1e-320)
plot(err('Q','Q',1e-16,1e-06),label='1')
plot(err('L','L',1e-14,1e-06),label='2')
plot(err('Q','L',1e-16,1e-14),label='3')
yscale('log')
ylim(1e-14,2.1)
legend();

In [ ]:
from scipy.sparse.linalg import expm_multiply
prop = lanczos_timeprop(H,phi0,12,1e-12)
HT_done = prop._step(0.,0.5)
print(HT_done)
phi2 = expm_multiply(-1j*HT_done*H,phi0)
np.allclose(prop.phia[0],phi2)

In [ ]:
%timeit lanczos_timeprop(H,phi0,12,1e-12)._step(0.,HT_done)
prop = lanczos_timeprop(H,phi0,12,1e-12)
%timeit prop._step(0.,HT_done)
%timeit phi2 = expm_multiply(-1j*HT_done*H,phi0)
%timeit sesolve(Hq,phi0q,[0,HT_done],[])